## Creating subagents

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """ Calculates the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """ Calculates the square of a number """
    return x ** 2


In [5]:
from langchain.agents import create_agent

subagent_1 = create_agent(
    model = "gpt-5-nano",
    tools = [square_root]
)

subagent_2 = create_agent(
    model = "gpt-5-nano",
    tools = [square]
)

In [7]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """ Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({'messages': [HumanMessage(content = f"Calculate the square root of {x}")]})
    return response['messages'][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """ Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({'messages': [HumanMessage(content = f"Calculate the square of {x}")]})
    return response['messages'][-1].content


# Creating the main agent

main_agent = create_agent(
    model = "gpt-5-nano",
    tools = [call_subagent_1, call_subagent_2],
    system_prompt = "You are a helpful assistant who can call subagents to calculate the square root or square of a number."
)

In [10]:
question = "What is the square root of 42?"

response = main_agent.invoke({"messages": HumanMessage(content = question)})

from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='What is the square root of 42?', additional_kwargs={}, response_metadata={}, id='86451ef9-4359-4b5c-934c-42e9684805ba'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 284, 'prompt_tokens': 202, 'total_tokens': 486, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpQyva7yQllIAUnlnTXW81rmYH7MS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b441f-9e7f-74a2-aed3-ad2514d22140-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 42}, 'id': 'call_rBxEhKlA61nvC1Q7buc3u6lk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 284, 'tota